In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import networkx as nx
import requests
import io

# Load the geochemical data
file_path = '/content/drive/MyDrive/Geochem_Split_2/Porphyry Cu-Mo-Au.csv'
data = pd.read_csv(file_path)

# Print the unique values in the 'DEPOSIT_TYPE' column
unique_deposit_types = data['DEPOSIT_TYPE'].unique()
print("Unique DEPOSIT_TYPE values:")
for deposit_type in unique_deposit_types:
    print(deposit_type)


Unique DEPOSIT_TYPE values:
High sulfidation Au-Ag
Lithocap alunite/High sulfidation Au-Ag
Lithocap kaolinite/High sulfidation Au-Ag
Porphyry/skarn Cu and Distal disseminated Au-Au
Porphyry Cu or High sulfidation Au-Ag
Porphyry/skarn Cu
Lithocap alunite
Polymetallic sulfide vein
Porphyry Mo
Intermediate sulfidation epithermal Au-Ag
Porphyry Cu
Polymetallic sulfide IS
Low sulfidation Au-Ag
High sulfidation Au-Ag (Cu)
Low sulfidation
Low sulfidation Sb
Low sulfidation Hg
High sulfidation Sulfur
Porphyry Cu (Au)
Polymetallic sulfide replacement
Skarn W
Polymetallic sulfide skarn/replacement
Polymetallic sulfide intermediate sulfidation
Distal Disseminated Ag-Au
Porphyry Cu-Mo
Polymetallic sulfide skarn
Distal disseminated Ag-Au


<ipython-input-2-4ac8297d6039>:13: DtypeWarning: Columns (2,3,4,5,14,19,21,26,27,32,33,34,35,36,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)


In [34]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Columns to be used as features for prediction
selected_columns = [
    'Ag_ppm', 'Al_pct', 'As_ppm', 'Au_ppm', 'B_ppm', 'Ba_ppm', 'Be_ppm', 'Bi_ppm', 'Br_ppm',
    'C_pct', 'Ca_pct', 'Cd_ppm', 'Ce_ppm', 'Cl_pct', 'Co_ppm', 'Cr_ppm', 'Cs_ppm', 'Cu_ppm',
    'Dy_ppm', 'Er_ppm', 'Eu_ppm', 'F_pct', 'Fe_pct', 'Ga_ppm', 'Gd_ppm', 'Ge_ppm', 'Hf_ppm',
    'Hg_ppm', 'Ho_ppm', 'In_ppm', 'Ir_ppm', 'K_pct', 'La_ppm', 'Li_ppm', 'Lu_ppm', 'Mg_pct',
    'Mn_pct', 'Mo_ppm', 'Na_pct', 'Nb_ppm', 'Nd_ppm', 'Ni_ppm', 'P_pct', 'Pb_ppm', 'Pd_ppm',
    'Pr_ppm', 'Pt_ppm', 'Rb_ppm', 'Re_ppm', 'Rh_ppm', 'Ru_ppm', 'S_pct', 'Sb_ppm', 'Sc_ppm',
    'Se_ppm', 'Si_pct', 'Sm_ppm', 'Sn_ppm', 'Sr_ppm', 'Ta_ppm', 'Tb_ppm', 'Te_ppm', 'Th_ppm',
    'Ti_pct', 'Tl_ppm', 'Tm_ppm', 'U_ppm', 'V_ppm', 'W_ppm', 'Y_ppm', 'Yb_ppm', 'Zn_ppm', 'Zr_ppm'
]


# Common Preprocessing Steps
def preprocess_data(data, selected_columns, rare_threshold=2):
    # Extract selected features and target
    features = data[selected_columns]
    target = data['DEPOSIT_TYPE']

    # Merge rare classes into "Other"
    target_counts = target.value_counts()
    rare_classes = target_counts[target_counts < rare_threshold].index
    target = target.replace(rare_classes, "Other")

    # Handle non-numeric columns by encoding them
    label_encoders = {}
    for column in features.select_dtypes(include=['object']).columns:
        le = LabelEncoder()
        features[column] = le.fit_transform(features[column])
        label_encoders[column] = le

    # Handle missing values (fill numeric columns with mean)
    numeric_features = features.select_dtypes(include=[np.number])
    features[numeric_features.columns] = numeric_features.fillna(numeric_features.mean())

    # Standardize numeric features
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)

    return features_scaled, target


# Split data into train/test/validation sets
def split_data(features, target, test_size=0.2, random_state=258):
    X_train, X_temp, y_train, y_temp = train_test_split(features, target, test_size=test_size, random_state=random_state)
    X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=random_state)
    return X_train, X_test, X_val, y_train, y_test, y_val


# Knowledge-based Feature Engineering for Model 2
def add_knowledge_features_model2(df):
    df['kb_porphyry_cu_potential'] = ((df['Cu_ppm'] > 200) & (df['Mo_ppm'] > 10) & (df['K_pct'] > 2)).astype(int)
    df['kb_skarn_related_potential'] = ((df['W_ppm'] > 10) | (df['Fe_pct'] > 10) | (df['Ca_pct'] > 5)).astype(int)
    df['kb_high_sulfidation_potential'] = ((df['Cu_ppm'] > 500) & (df['Au_ppm'] > 0.1)).astype(int)
    df['kb_low_sulfidation_potential'] = ((df['Ag_ppm'] > 50) & (df['Au_ppm'] > 0.05)).astype(int)
    return df


# Knowledge-based Feature Engineering for Model 3
def add_knowledge_features_model3(df):
    df = add_knowledge_features_model2(df)
    df['porphyry_cu_mo_potential'] = ((df['Cu_ppm'] > 300) & (df['Mo_ppm'] > 15)).astype(int)
    df['lithocap_potential'] = ((df['Al_pct'] > 5) & (df['K_pct'] > 1)).astype(int)
    df['polymetallic_sulfide_potential'] = ((df['Pb_ppm'] > 100) & (df['Zn_ppm'] > 100) & (df['Ag_ppm'] > 10)).astype(int)
    df['intermediate_sulfidation_potential'] = ((df['Zn_ppm'] > 50) & (df['Au_ppm'] > 0.05) & (df['Cu_ppm'] > 100)).astype(int)
    df['distal_disseminated_potential'] = ((df['Ag_ppm'] > 30) & (df['As_ppm'] > 50)).astype(int)
    df['high_alumina_alteration'] = ((df['Al_pct'] > 8) & (df['Si_pct'] > 10)).astype(int)
    df['low_sulfidation_sb_potential'] = (df['Sb_ppm'] > 10).astype(int)
    df['skarn_w_potential'] = ((df['W_ppm'] > 20) & (df['Fe_pct'] > 5)).astype(int)
    df['high_sulfidation_sulfur_potential'] = (df['S_pct'] > 5).astype(int)
    df['porphyry_cu_au_potential'] = ((df['Cu_ppm'] > 300) & (df['Au_ppm'] > 0.15)).astype(int)
    return df

# Feature Importance Evaluation with Prefixed Feature Names
def evaluate_model_with_prefixed_features(
    model, X_train, y_train_encoded, X_test, y_test_encoded, X_val, y_val_encoded, feature_names=None
):
    # Evaluate on training set
    y_train_pred = model.predict(X_train)
    train_accuracy = accuracy_score(y_train_encoded, y_train_pred)
    print(f"Model Accuracy (Training Set): {train_accuracy}")
    print("Classification Report (Training Set):")
    print(
        classification_report(
            y_train_encoded, y_train_pred,
            labels=np.unique(y_train_encoded),
            target_names=target_encoder.inverse_transform(np.unique(y_train_encoded)),
        )
    )

    # Evaluate on test set
    y_test_pred = model.predict(X_test)
    test_accuracy = accuracy_score(y_test_encoded, y_test_pred)
    print(f"Model Accuracy (Test Set): {test_accuracy}")
    print("Classification Report (Test Set):")
    print(
        classification_report(
            y_test_encoded, y_test_pred,
            labels=np.unique(y_test_encoded),
            target_names=target_encoder.inverse_transform(np.unique(y_test_encoded)),
        )
    )

    # Evaluate on validation set
    y_val_pred = model.predict(X_val)
    val_accuracy = accuracy_score(y_val_encoded, y_val_pred)
    print(f"Model Accuracy (Validation Set): {val_accuracy}")
    print("Classification Report (Validation Set):")
    print(
        classification_report(
            y_val_encoded, y_val_pred,
            labels=np.unique(y_val_encoded),
            target_names=target_encoder.inverse_transform(np.unique(y_val_encoded)),
        )
    )

    # Feature importance (if applicable)
    if feature_names is not None and hasattr(model, "feature_importances_"):
        feature_importances = pd.Series(model.feature_importances_, index=feature_names).sort_values(ascending=False)
        print("Top 10 Feature Importances:")
        print(feature_importances.head(10))


In [35]:
# Model 1: No Knowledge-Based Features
features_scaled, target = preprocess_data(data, selected_columns)
X_train, X_test, X_val, y_train, y_test, y_val = split_data(features_scaled, target)

# Encode targets
target_encoder = LabelEncoder()
y_train_encoded = target_encoder.fit_transform(y_train)
y_test_encoded = target_encoder.transform(y_test)
y_val_encoded = target_encoder.transform(y_val)

# Train RandomForest model
rf_clf = RandomForestClassifier(n_estimators=100, random_state=33, class_weight='balanced')
rf_clf.fit(X_train, y_train_encoded)

# Evaluate Model 1
evaluate_model_with_prefixed_features(
    rf_clf, X_train, y_train_encoded, X_test, y_test_encoded, X_val, y_val_encoded, feature_names=selected_columns
)


<ipython-input-34-bede8cd4d32b>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features[numeric_features.columns] = numeric_features.fillna(numeric_features.mean())
/usr/local/lib/python3.11/dist-packages/sklearn/utils/extmath.py:1101: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.11/dist-packages/sklearn/utils/extmath.py:1106: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.11/dist-packages/sklearn/utils/extmath.py:1126: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


Model Accuracy (Training Set): 1.0
Classification Report (Training Set):
                                                 precision    recall  f1-score   support

                      Distal Disseminated Ag-Au       1.00      1.00      1.00         8
                      Distal disseminated Ag-Au       1.00      1.00      1.00         5
                         High sulfidation Au-Ag       1.00      1.00      1.00       396
                    High sulfidation Au-Ag (Cu)       1.00      1.00      1.00        16
      Intermediate sulfidation epithermal Au-Ag       1.00      1.00      1.00         1
                               Lithocap alunite       1.00      1.00      1.00       107
        Lithocap alunite/High sulfidation Au-Ag       1.00      1.00      1.00       108
      Lithocap kaolinite/High sulfidation Au-Ag       1.00      1.00      1.00        57
                                Low sulfidation       1.00      1.00      1.00       110
                          Low sulfid

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

In [36]:
# Model 2: Add Limited Knowledge-Based Features
features_knowledge = add_knowledge_features_model2(pd.DataFrame(features_scaled, columns=selected_columns))
combined_features = np.concatenate([features_scaled, features_knowledge.values], axis=1)
X_train, X_test, X_val, y_train, y_test, y_val = split_data(combined_features, target)

# Train RandomForest model
rf_clf.fit(X_train, y_train_encoded)

# Evaluate Model 2
feature_names_model2 = [f"original_{col}" for col in selected_columns] + [f"knowledge_{col}" for col in features_knowledge.columns]
evaluate_model_with_prefixed_features(
    rf_clf, X_train, y_train_encoded, X_test, y_test_encoded, X_val, y_val_encoded, feature_names=feature_names_model2
)


Model Accuracy (Training Set): 1.0
Classification Report (Training Set):
                                                 precision    recall  f1-score   support

                      Distal Disseminated Ag-Au       1.00      1.00      1.00         8
                      Distal disseminated Ag-Au       1.00      1.00      1.00         5
                         High sulfidation Au-Ag       1.00      1.00      1.00       396
                    High sulfidation Au-Ag (Cu)       1.00      1.00      1.00        16
      Intermediate sulfidation epithermal Au-Ag       1.00      1.00      1.00         1
                               Lithocap alunite       1.00      1.00      1.00       107
        Lithocap alunite/High sulfidation Au-Ag       1.00      1.00      1.00       108
      Lithocap kaolinite/High sulfidation Au-Ag       1.00      1.00      1.00        57
                                Low sulfidation       1.00      1.00      1.00       110
                          Low sulfid

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

In [37]:
# Model 3: Add All Knowledge-Based Features
features_knowledge = add_knowledge_features_model3(pd.DataFrame(features_scaled, columns=selected_columns))
combined_features = np.concatenate([features_scaled, features_knowledge.values], axis=1)
X_train, X_test, X_val, y_train, y_test, y_val = split_data(combined_features, target)

# Train RandomForest model
rf_clf.fit(X_train, y_train_encoded)

# Evaluate Model 3
feature_names_model3 = [f"original_{col}" for col in selected_columns] + [f"knowledge_{col}" for col in features_knowledge.columns]
evaluate_model_with_prefixed_features(
    rf_clf, X_train, y_train_encoded, X_test, y_test_encoded, X_val, y_val_encoded, feature_names=feature_names_model3
)

Model Accuracy (Training Set): 1.0
Classification Report (Training Set):
                                                 precision    recall  f1-score   support

                      Distal Disseminated Ag-Au       1.00      1.00      1.00         8
                      Distal disseminated Ag-Au       1.00      1.00      1.00         5
                         High sulfidation Au-Ag       1.00      1.00      1.00       396
                    High sulfidation Au-Ag (Cu)       1.00      1.00      1.00        16
      Intermediate sulfidation epithermal Au-Ag       1.00      1.00      1.00         1
                               Lithocap alunite       1.00      1.00      1.00       107
        Lithocap alunite/High sulfidation Au-Ag       1.00      1.00      1.00       108
      Lithocap kaolinite/High sulfidation Au-Ag       1.00      1.00      1.00        57
                                Low sulfidation       1.00      1.00      1.00       110
                          Low sulfid

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m